<a href="https://colab.research.google.com/github/mille-s/ACLPUB-sm/blob/main/aclpub2_SM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run for getting yml files and generating proceedings
! git clone https://github.com/rycolab/aclpub2.git
! pip install -r '/content/aclpub2/softconf/requirements.txt'

# Get yml files from SoftConf

In [ ]:
import codecs

# Create config file

config_file_path = '/content/aclpub2/softconf/config.json'
script_path = '/content/aclpub2/softconf/softconf2aclpub.py'

# Read original config file
config_file = open(config_file_path, 'r')
lines_config_file = config_file.readlines()

# Update config file
with codecs.open(config_file_path, 'w', 'utf-8') as f:
  for line in lines_config_file:
    if line.startswith('"conf"'):
      f.write('"conf" : "inlg2023",\n')
    elif line.startswith('"track"'):
      f.write('"track" : "GenerationChallenges",\n')
    elif line.startswith('"username"'):
      f.write('"username" : "smille",\n')
    elif line.startswith('"password"'):
      f.write('"password" : "xxxxx",\n')
    elif line.startswith('"service_program_committee"'):
      f.write('"service_program_committee" : "https://softconf.com/n/inlg2023/cgi-bin/scmd.cgi?scmd=webservice&token=2T-D6D9H8G7E6A3B3B3C9A7",\n')
    elif line.startswith('"service_papers"'):
      f.write('"service_papers" : "https://softconf.com/n/inlg2023/cgi-bin/scmd.cgi?scmd=webservice&token=1T-A2B6C3H3A2J4B2B3D9G6"\n')
    else:
      f.write(line)

# Read original script
script_file = open(script_path, 'r')
lines_script_file = script_file.readlines()

# Update script file
with codecs.open(script_path, 'w', 'utf-8') as f:
  for line in lines_script_file:
    if line.startswith('with open("config.json") as f:'):
      f.write('with open("/content/aclpub2/softconf/config.json") as f:\n')
    elif line.startswith('CONF_URL'):
      f.write('CONF_URL = f"{SOFTCONF_URL}/n/{config[\'conf\']}/track/{config[\'track\']}/"\n')
    elif line.startswith('get_conference_details()'):
      f.write('#get_conference_details()\n')
    elif line.startswith('get_files()'):
      f.write('#get_files()\n')
    else:
      f.write(line)

In [ ]:
# Run export script
# I could get the PC and the papers automatically, by commenting the "get_conference_details()" and "get_files()" at the end of softconf2aclpub.py, which seem to give errors

path_script = '/content/aclpub2/softconf/softconf2aclpub.py'
!python {path_script}

#Unzip uploaded file

In [ ]:
# First drag and drop zip file with all yml files and papers to the local drive, then run this cell
# See https://github.com/rycolab/aclpub2/tree/main#Testing-the-tool-to-generate-your-proceedings for details of the contents of the zip
! unzip /content/proceedings.zip -d /content/proceedings

# Generate proceedings

In [ ]:
# Install tex compiler
! pip install pdflatex
! pip install pypdf
! pip install roman
! sudo apt-get install texlive-latex-base texlive-latex-recommended texlive-latex-extra texlive-fonts-recommended texlive-fonts-extra texlive-bibtex-extra

In [ ]:
! whereis latex

latex: /usr/bin/latex /usr/share/man/man1/latex.1.gz


In [ ]:
# import os
# os.environ['PATH'] += "/usr/bin/latex"

In [ ]:
import codecs
import os
from pdflatex import PDFLaTeX

# Read main script
main_path = '/content/aclpub2/bin/generate'
main_file = open(main_path, 'r')
lines_main_file = main_file.readlines()
# Update main script to add path so it can import "local" modules
with codecs.open(main_path, 'w', 'utf-8') as f:
  for line in lines_main_file:
    if line.startswith('import argparse'):
      f.write('import argparse\nimport sys\nsys.path.insert(0,"/content/aclpub2")\n')
    else:
      f.write(line)

# Read main Py script
main_Py_path = '/content/aclpub2/aclpub2/generate.py'
main_Py_file = open(main_Py_path, 'r')
lines_main_Py_file = main_Py_file.readlines()
# Update main Python script to update deprecated parts
with codecs.open(main_Py_path, 'w', 'utf-8') as f:
  for line in lines_main_Py_file:
    if line.startswith(r'from PyPDF2 import PdfFileReader'):
      f.write(r'from pypdf import PdfReader')
      f.write('\n')
    elif line.startswith(r'        pdf = PdfFileReader(str(pdf_path))'):
      f.write(r'        pdf = PdfReader(str(pdf_path))')
      f.write('\n')
    elif line.startswith(r'        paper["num_pages"] = pdf.getNumPages()'):
      f.write(r'        paper["num_pages"] = len(pdf.pages)')
      f.write('\n')
    elif line.startswith(r'        paper["end_page"] = page + pdf.getNumPages() - 1'):
      f.write(r'        paper["end_page"] = page + len(pdf.pages) - 1')
      f.write('\n')
    elif line.startswith(r'        page += pdf.getNumPages()'):
      f.write(r'        page += len(pdf.pages)')
      f.write('\n')
    else:
      f.write(line)

# Read latex template
template_path = '/content/aclpub2/aclpub2/templates/proceedings.tex'
template_file = open(template_path, 'r')
lines_template_file = template_file.readlines()
# Update main script to add path so it can import "local" modules
with codecs.open(template_path, 'w', 'utf-8') as f:
  for line in lines_template_file:
    if line.startswith(r'%\usepackage[T1]{fontenc}'):
      f.write(r'\usepackage[T1]{fontenc}')
      f.write('\n')
    elif line.startswith(r'\usepackage[T5]{fontenc}'):
      f.write(r'%\usepackage[T5]{fontenc}')
      f.write('\n')
    elif line.startswith(r'\usepackage[english,latin]{babel}'):
      f.write(r'\usepackage[english]{babel}')
      f.write('\n')
    else:
      f.write(line)

# import sys
# sys.path.insert(0,'/content/aclpub2')

# Generates the proceedings.
# ! /content/aclpub2/bin/generate /content/aclpub2/examples/sigdial --proceedings --overwrite
! /content/aclpub2/bin/generate /content/proceedings --proceedings --overwrite

# # Generates both and overwrites the existing contents of the build directory.
# ! '/content/aclpub2/bin/generate' examples/sigdial --proceedings --handbook --overwrite

In [ ]:
!zip -r genchal23-proceedings.zip /content/build/